# Import

In [87]:
import pandas as pd

In [88]:
import matplotlib.pyplot as plt

# 한글 폰트 지정
plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['axes.unicode_minus'] =False

font_path = "C:/Windows/Fonts/malgun.ttf"

# Load Data

In [89]:
train_df = pd.read_csv("../../data/train.csv")
test_df = pd.read_csv("../../data/test.csv")  
submission = pd.read_csv("../../data/sample_submission.csv")  

In [90]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54609 entries, 0 to 54608
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      54609 non-null  object
 1   분류      54609 non-null  object
 2   제목      54609 non-null  object
 3   키워드     54609 non-null  object
dtypes: object(4)
memory usage: 1.7+ MB


In [91]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23405 entries, 0 to 23404
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      23405 non-null  object
 1   제목      23405 non-null  object
 2   키워드     23405 non-null  object
dtypes: object(3)
memory usage: 548.7+ KB


In [92]:
submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23405 entries, 0 to 23404
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      23405 non-null  object
 1   분류      23405 non-null  object
dtypes: object(2)
memory usage: 365.8+ KB


In [93]:
len(train_df['분류'].value_counts())

56

타겟변수 -> '분류'이고 종류 56가지

In [94]:
test_df.head()

,ID,제목,키워드
0,TEST_00000,[부고] 김태수씨 별세 외,"김태수,별세,김태수씨,서울,광남초등학,교장,별세,김윤정,이노코리아,대표,희정,한성대..."
1,TEST_00001,"신규 확진 나흘째 세자릿수... 방역당국, 핼러윈 풍선효과 차단 총력","신규,확진,나흘,세자릿수,방역당국,핼러윈,풍선,효과,차단,총력,감염증,신종,코로나바..."
2,TEST_00002,"[서경이 만난 사람] 전해철 장관 ""재정분권 강화 '지방자치 2.0 시대' 마중물 ...","전해철,장관,재정,분권,강화,지방자치,2.0,시대,마중물,마련,장관,전해철,행정안전..."
3,TEST_00003,"용인시, 12일 '장애인 구인 구직 만남의 날' 채용 행사","용인시,구인,장애인,구직,만남,채용,행사,노호근,용인특례시,장애인,취업,지원,대회의..."
4,TEST_00004,지자체 벽 터 경기지역 산단 활성화 모색,"지자체,경기,북동부,지역,산업단지,혁신단위,설정,전략,지역,연계,특성,제시,경기도경..."


영어 번역 내용 제거

In [95]:
# '키워드' 열에서 '기사'와 '구글' 두 단어를 모두 포함하는 행을 찾는 조건
condition = train_df['키워드'].apply(lambda x: all(word in x for word in ['기사', '구글']))

# 조건을 만족하는 행들로 새로운 데이터프레임 생성
filtered_df = train_df[condition].copy()

# '기사, 구글' 순서로 단어가 나오는 부분을 찾아 삭제하는 함수 정의
def remove_keywords(text):
    keywords = ['기사', '구글']
    start_index = 0
    for keyword in keywords:
        start_index = text.find(keyword, start_index)
        if start_index == -1:
            return text
        start_index += len(keyword)
    return text[:text.find('기사')].strip()

# 각 행의 '키워드' 값에서 '기사' 단어부터 마지막 단어까지 삭제
filtered_df['키워드'] = filtered_df['키워드'].apply(remove_keywords)

In [96]:
filtered_df.iloc[0]['키워드']

'이상일,용인,특례,시장,참석,민주평통,경기,지역,의장,표창,수여식,시장,민주평통,평화,수호,선도,활동,감사,강조,이상일,용인,특례,시장,에이스홀,시청,민주평화통일자문회,수여식,경기,지역,의장,표창,참석,관계자들,격려,자리,시장,홍승표,민주평통,경기,지역,회의,부의장,용인시협의회장,추상구,민주평통,용인시,협의회장,자문위원,민주평통,경기,지역,자문,위원,400여명,참석,시장,민주평화통일자문회,경기지역회의,안보태세,수호,한반도,평화,선도,활동,감사,시장,북한,도발,주시,차례,특강,정세,한반도,안보,강화,강조,내년,민주평통,활동,응원,차원,지원,수여식,이날,표창,임명숙,자문위원,장석순,용인시협의회,자문,위원,자치분권,정지혜,용인시,자치,분권,민간협력팀장,62명,유공자,의장,표창,용인시협의회,평화통일포럼,통일공감사업,진행,홍보,평화,통일,정책,노력,유공,기관,표창,대표,이소연,뉴코리아여성연합,통일,페이스,주제,특강,진행,'

In [97]:
len(filtered_df['분류'].value_counts())

47

In [98]:
filtered_df['분류'].value_counts()

지역               1594
IT_과학:인터넷_SNS      59
사회:장애인             38
사회:사회일반            36
사회:교육_시험           35
경제:취업_창업           34
경제:반도체             34
문화:전시_공연           30
경제:부동산             24
스포츠:올림픽_아시안게임      22
정치:행정_자치           22
경제:경제일반            21
IT_과학:콘텐츠          20
사회:의료_건강           20
사회:여성              18
경제:자동차             18
정치:국회_정당           18
경제:무역              17
사회:환경              16
정치:정치일반            15
문화:문화일반            14
경제:유통              13
IT_과학:모바일          12
경제:금융_재테크          11
문화:출판              11
문화:학술_문화재          10
경제:서비스_쇼핑          10
문화:방송_연예            9
문화:미술_건축            9
스포츠:축구              9
정치:선거               7
문화:음악               5
사회:노동_복지            5
정치:청와대              4
IT_과학:보안            4
IT_과학:IT_과학일반       4
스포츠:월드컵             3
사회:사건_사고            3
국제                  3
경제:자원               3
문화:종교               2
정치:북한               2
IT_과학:과학            1
사회:미디어              1
문화:요리_여행            1
경제:산업_기업  

실제 제거하여 데이터셋에 반영

In [99]:
# '키워드' 열에서 '기사'와 '구글' 두 단어를 모두 포함하는 행을 찾는 조건
condition_train = train_df['키워드'].apply(lambda x: all(word in x for word in ['기사', '구글']))
condition_test = test_df['키워드'].apply(lambda x: all(word in x for word in ['기사', '구글']))

# 조건을 만족하는 행들로 새로운 데이터프레임 생성
filtered_train_df = train_df[condition_train].copy()
filtered_test_df = test_df[condition_test].copy()

# '기사, 구글' 순서로 단어가 나오는 부분을 찾아 삭제하는 함수 정의
def remove_keywords(text):
    keywords = ['기사', '구글']
    start_index = 0
    for keyword in keywords:
        start_index = text.find(keyword, start_index)
        if start_index == -1:
            return text
        start_index += len(keyword)
    return text[:text.find('기사')].strip()

# 각 행의 '키워드' 값에서 '기사' 단어부터 마지막 단어까지 삭제
filtered_train_df['키워드'] = filtered_train_df['키워드'].apply(remove_keywords)
filtered_test_df['키워드'] = filtered_test_df['키워드'].apply(remove_keywords)

# 수정된 '키워드' 값을 원래 데이터프레임에 반영
train_df.loc[condition_train, '키워드'] = filtered_train_df['키워드']
test_df.loc[condition_test, '키워드'] = filtered_test_df['키워드']

In [100]:
filtered_df.iloc[0]['키워드']

'이상일,용인,특례,시장,참석,민주평통,경기,지역,의장,표창,수여식,시장,민주평통,평화,수호,선도,활동,감사,강조,이상일,용인,특례,시장,에이스홀,시청,민주평화통일자문회,수여식,경기,지역,의장,표창,참석,관계자들,격려,자리,시장,홍승표,민주평통,경기,지역,회의,부의장,용인시협의회장,추상구,민주평통,용인시,협의회장,자문위원,민주평통,경기,지역,자문,위원,400여명,참석,시장,민주평화통일자문회,경기지역회의,안보태세,수호,한반도,평화,선도,활동,감사,시장,북한,도발,주시,차례,특강,정세,한반도,안보,강화,강조,내년,민주평통,활동,응원,차원,지원,수여식,이날,표창,임명숙,자문위원,장석순,용인시협의회,자문,위원,자치분권,정지혜,용인시,자치,분권,민간협력팀장,62명,유공자,의장,표창,용인시협의회,평화통일포럼,통일공감사업,진행,홍보,평화,통일,정책,노력,유공,기관,표창,대표,이소연,뉴코리아여성연합,통일,페이스,주제,특강,진행,'

In [101]:
# # 수정된 '키워드' 값을 원래 데이터프레임에 반영
# train_df.loc[condition_train, '키워드'] = filtered_train_df['키워드']
# test_df.loc[condition_test, '키워드'] = filtered_test_df['키워드']

# # 수정된 데이터프레임을 CSV 파일로 저장
# train_df.to_csv('train_df_translate_del.csv', index=False, encoding='utf-8-sig')
# test_df.to_csv('test_df_translate_del.csv', index=False, encoding='utf-8-sig')

## 전처리

In [102]:
import re
from collections import Counter

# 불용어 제거
removed_keywords = []

def remove_invalid_keywords(keywords):
    pattern = re.compile(
        r'[A-Za-z]+[가-힣\u4E00-\u9FFF]+|'  # 영어+한글
        r'[0-9]+[가-힣\u4E00-\u9FFF]+|'     # 숫자+한글
        r'[0-9]+[A-Za-z]+|'                 # 숫자+영어
        r'[가-힣]+[A-Za-z\u4E00-\u9FFF]+|'  # 한글+영어
        r'[가-힣]+[0-9]+|'                  # 한글+숫자
        r'[A-Za-z]+[0-9]+|'                 # 영어+숫자
        r'[\u4E00-\u9FFF]+|'                # 한자
        r'[0-9]+(\.[0-9]+)?%|'              # 숫자+퍼센트
        r'www\.[^\s]+|'                    # www로 시작하는 단어
        r'http[^\s]*|'                     # http로 시작하는 단어
        r'[0-9]+'                           # 숫자
    )
    valid_keywords = []
    for word in keywords.split(','):
        word = word.strip()
        if not word or pattern.match(word):  # 공백이거나 패턴에 맞는 단어 제거
            if word not in ['코로나19', 'QR코드', 'RE100']:  # 예외 처리
                removed_keywords.append(word)
            else:
                valid_keywords.append(word)
        else:
            valid_keywords.append(word)
    return ', '.join(valid_keywords)

# 원본 데이터에서 해당 키워드들을 제거
train_df['키워드'] = train_df['키워드'].apply(remove_invalid_keywords)
test_df['키워드'] = test_df['키워드'].apply(remove_invalid_keywords)

# 제거된 단어들 출력
print("\n제거된 단어들:")
print(removed_keywords[:10])


제거된 단어들:
['18일', '15일', '7월', '8월', '50%', '22일', '20명', '10명', '70명', '36종류']


In [103]:
# 제거된 단어들의 빈도 계산 및 상위 50개 출력
keyword_counter = Counter(removed_keywords)
top_50_keywords = keyword_counter.most_common(3000)

print("\n제거된 단어들 상위 50개:")
for keyword, count in top_50_keywords:
    print(f"{keyword}: {count}")


제거된 단어들 상위 50개:
A씨: 13186
: 7207
1만: 6451
SK하이닉스: 5605
1년: 4812
3년: 4033
2년: 3948
B씨: 3770
6월: 3513
1명: 3455
12월: 3436
7월: 3407
3일: 3389
3월: 3347
1억: 3274
1천: 3178
2만: 3122
9월: 3099
4월: 3056
2명: 3024
3만: 2953
10월: 2874
5월: 2849
8월: 2814
1: 2791
3명: 2749
5년: 2740
7일: 2740
22일: 2734
23일: 2725
27일: 2708
8일: 2693
14일: 2684
11월: 2684
30일: 2668
28일: 2663
13일: 2641
15일: 2606
20일: 2600
1일: 2590
26일: 2579
18일: 2577
3개: 2561
4일: 2510
5일: 2507
17일: 2503
12일: 2501
25일: 2478
19일: 2464
10일: 2415
50%: 2409
6일: 2367
10년: 2357
21일: 2334
29일: 2314
24일: 2314
9일: 2260
1월: 2200
2022년: 2194
10만: 2161
2019년: 2144
16일: 2138
4개: 2131
1위: 2111
2일: 2102
2억: 2097
100만: 2053
11일: 2048
2월: 1993
10%: 1984
2021년: 1964
4명: 1947
3억: 1932
2020년: 1908
4만: 1849
5명: 1801
10명: 1795
30%: 1779
4년: 1757
9시: 1752
2026년: 1718
1인: 1708
5개: 1691
5만: 1689
2018년: 1659
5억: 1644
31일: 1640
100%: 1628
6명: 1615
6시: 1610
20%: 1594
2천: 1550
8억: 1538
6개월: 1535
10억: 1508
10시: 1497
2025년: 1470
4억: 1442
2차: 1371
2: 1369
2024년: 1363
2개: 1296
3

'경기용인 -> 경기용, [전문] -> 전문' 이렇게 들어간 값이 있어서 제거

In [104]:
# '경기용'과 '전문' 단어 제거 함수
def remove_keywords(keywords, words_to_remove):
    keywords_list = keywords.split(', ')
    filtered_keywords = [word for word in keywords_list if word not in words_to_remove]
    return ', '.join(filtered_keywords)

# 제거할 단어 목록
words_to_remove = ['경기용', '전문']

# '키워드' 열 업데이트
train_df['키워드'] = train_df['키워드'].apply(lambda x: remove_keywords(x, words_to_remove))
test_df['키워드'] = test_df['키워드'].apply(lambda x: remove_keywords(x, words_to_remove))

In [105]:
# 정확히 '경기용' 단어를 포함하고 있는 행 필터링 / 경기용인플랫폼 이런 단어는 찾지 않음
filtered_df = train_df[train_df['키워드'].str.contains(r'\b경기용\b', na=False)]

# 결과 출력
print(filtered_df)

Empty DataFrame
Columns: [ID, 분류, 제목, 키워드]
Index: []


In [106]:
# 정확히 '경기용' 단어를 포함하고 있는 행 필터링 / 경기용인플랫폼 이런 단어는 찾지 않음
filtered_df = train_df[train_df['키워드'].str.contains(r'https', na=False)]

# 결과 출력
print(filtered_df)

Empty DataFrame
Columns: [ID, 분류, 제목, 키워드]
Index: []


In [107]:
# 정확히 일치하는 행 필터링
filtered_df = train_df[train_df['키워드'].str.contains(r'DX-', na=False)]

# 결과 출력
filtered_df.iloc[0]['제목']

'도깨비 아이언 DX-109P, 현존 최고 고반발 소재로 비거리 늘려'

In [108]:
# 정확히 일치하는 행 필터링
filtered_df = train_df[train_df['키워드'].str.contains(r'DX-', na=False)]

# 결과 출력
filtered_df.iloc[0]['키워드']

'DX-, 도깨비, 아이언, DX-, 현존, 최고, 고반발, 소재, 비거리, 골프시장, 니즈, 충족, 사업, 확장, 신제품, 출시, 제품, 스윙, 중량급, 거리, 완성, 드라이버, DX-, 아이언, 쌍둥이, 일명, DX-, 아이언, 현존, 최고, 마레이징, 고반발, 소재, 페이스, 사용, 비거리, 탄도, 구사, 아이언, 쌍둥이, 길이, 아이언, 아이언, 시니어, 라인, 레귤러, 라인, 스탠더드, 라인, 세분화, 업그레이드, 제품, 추가, 준비, 도깨비골프, 회사, 규모, 경기, 성남, 분당구, 궁내동, 용인시, 동천역, 확장, 사명, 변경, 강남, 라운지, 오픈'

In [109]:
# 정확히 일치하는 행 필터링
filtered_df = train_df[train_df['키워드'].str.contains(r'ᅳᆯ', na=False)]

# 결과 출력
filtered_df.iloc[0]['제목']

'"김동성, 양육비 8천만원 보냈는데 배드파더스 등재 평생 꼬리표" 인민정의 호소 [전문]'

In [110]:
# 정확히 일치하는 행 필터링
filtered_df = train_df[train_df['키워드'].str.contains(r'ᅳᆯ', na=False)]

# 결과 출력
filtered_df.iloc[0]['키워드']

'김동성, 양육비, 배드파더스, 등재, 평생, 꼬리표, 인민정, 호소, 김동성, 쇼트, 트랙, 국가, 대표, 극단적, 선택, 인민정, 여자, 친구, 인민정, 자신, SNS, 심경, 인스타그램, 저, ᆼ글맘, 자신, 처지, 공개, 우울증, 공황, 장애, 김동성, 공평, 상황들, 뇌리, 장문, 김동성, ᅥᆫ만원정도, 살아야, 로, 고, 그, 꼬리표는, 평생, 달고, 사, 토로, 김동성씨, 방송, ᅳᆯ, 도, 아무것, ᅡᄀ, 스, 써, 여, ᆷᄒ, ᅩᄅ, 데, 극단적, ᅳᆯ, 할, 수밖에, 어, 바닥, 어, 이, 지옥, 상황, 결과, 호소, 김동성, 지난달, 용인시, 상현동, 자택, 극단적, 선택, 시도, 신고, 출동, 대원, 병원, 이송, 생명, 지장, ᅦᄀ, ᅳᆫ, 질타, 않으면, ᆯᄋ, ᅳᆫ데, 저는, 제가, 눈을, 뜨고, 있는, 한, 어떠한, 상황에서도, 발벗고, 나서지, 아, 일분이, 가, 이, 이, 이, 소중하기ᄋ, ᅳᆫ, 야, 설명, 배드파더스, 경제활동, 악의적, 양육비, 비양육자, 채찍질, 소득, 수입, 재산, 비양육자, 배드파더스, 낙인, 합리적, 개인적, 생각, 아내, 이혼, 양육비, 논란, 김동성, 이혼, 프로그램, 출연, 인민정, 인스타, 그램, 싱글맘, 양육자와, 비양육자, ᆼ글맘입니다, 지금껏, 양육비ᄂ, 이엄마입니다, 도, ᅳᆯ, 걸까, 이, 이, 서, 이, 벌었습니다, 도, 아, ᅳᆯ, ᅳᆯ, 김동성씨ᄂ, 고, 랴, 로, 이, ᅥᆫ만원정도입니다, 도, 로, 고, 그, 꼬리표는, 평생, 달고, 사, 남자친구, ᅦᄀ, ᅳᆫ, 고, 느, ᆯᄋ,

제거된 상위 10개만 다시 확인

In [111]:
# 제거된 단어들의 빈도 계산 및 상위 10개 출력
keyword_counter = Counter(removed_keywords)
top_10_keywords = keyword_counter.most_common(10)

print("\n제거된 단어들 상위 10개:")
for keyword, count in top_10_keywords:
    print(f"{keyword}: {count}")


제거된 단어들 상위 10개:
A씨: 13186
: 7207
1만: 6451
SK하이닉스: 5605
1년: 4812
3년: 4033
2년: 3948
B씨: 3770
6월: 3513
1명: 3455


## 대분류 기준으로 공통단어 제거

In [112]:
# '분류' 열에서 앞부분만 추출하여 '분류_대분류'라는 새로운 열에 저장
train_df['분류_대분류'] = train_df['분류'].apply(lambda x: x.split(':')[0])

# 결과 확인
train_df['분류_대분류'].value_counts()

지역       26950
경제       10534
사회        8245
정치        2521
문화        2500
스포츠       2035
IT_과학     1487
국제         337
Name: 분류_대분류, dtype: int64

In [113]:
len(train_df)

54609

In [114]:
from collections import Counter

def find_common_words_and_remove(num_categories=8, top_n=100, com_counts=10, train_df=train_df, test_df=test_df):
    # '분류_대분류'의 각 범주에 속하는 단어들을 추출
    category_words = {category: [] for category in train_df["분류_대분류"].unique()}

    for category in category_words.keys():
        words = train_df.loc[train_df["분류_대분류"] == category, "키워드"].apply(lambda x: x.split(',')).tolist()
        category_words[category] = [word for sublist in words for word in sublist]

    # 각 범주에 속하는 단어들을 카운팅
    word_counts = {category: Counter(words) for category, words in category_words.items()}

    # 지정된 범주 갯수에 속하는 단어들을 찾기
    common_words = set()
    for word in word_counts[list(word_counts.keys())[0]].keys():
        count = sum(1 for category in word_counts.keys() if word in word_counts[category] and word_counts[category][word] >= com_counts)
        if count == num_categories:
            common_words.add(word)

    # 지정된 범주 갯수에 속하는 단어와 그 갯수를 계산
    common_word_counts = {word: sum(word_counts[category][word] for category in word_counts.keys()) for word in common_words}

    # 단어들을 갯수 기준으로 정렬하고 상위 N개를 선택
    top_common_words = sorted(common_word_counts.items(), key=lambda x: x[1], reverse=True)[:top_n]
    print(f"상위 {top_n}개의 단어 확인")

    # 상위 N개의 단어와 그 갯수, 그리고 각 범주에서의 갯수를 출력
    for word, total_count in top_common_words:
        category_counts = {category: word_counts[category][word] for category in word_counts.keys()}
        print(f"단어: {word}, 총 갯수: {total_count}, 각 범주에서의 갯수: {category_counts}")
        
    # 총 몇 개의 단어가 겹치는지 출력
    print(f"총 {len(common_words)}개의 단어가 겹칩니다.")

    # top_n 기준으로 겹치는 단어들을 '키워드' 열에서 제거하고 새로운 열에 저장
    top_common_words_set = set(word for word, _ in top_common_words)
    
    def remove_top_common_words(keywords):
        return ', '.join([word for word in keywords.split(',') if word not in top_common_words_set])

    train_df['키워드'] = train_df['키워드'].apply(remove_top_common_words)
    test_df['키워드'] = test_df['키워드'].apply(remove_top_common_words)

num_categories : 공통단어로 간주할 범주의 수  
top_n : 상위 N개의 공통 단어를 선택  
com_counts : 범주에서 단어가 공통 단어로 간주되기 위해 나타나야 하는 최소 횟수를 지정  

In [115]:
find_common_words_and_remove(num_categories=8, com_counts=100) # top_n은 default인 100 그대로 사용

상위 100개의 단어 확인
단어:  용인시, 총 갯수: 90817, 각 범주에서의 갯수: {'문화': 3624, '지역': 49123, '국제': 454, '정치': 3783, '경제': 17250, '사회': 11304, '스포츠': 2574, 'IT_과학': 2705}
단어:  용인, 총 갯수: 65977, 각 범주에서의 갯수: {'문화': 3297, '지역': 33651, '국제': 255, '정치': 3649, '경제': 19073, '사회': 3757, '스포츠': 1164, 'IT_과학': 1131}
단어:  지역, 총 갯수: 52102, 각 범주에서의 갯수: {'문화': 1356, '지역': 28457, '국제': 123, '정치': 3156, '경제': 12831, '사회': 5113, '스포츠': 171, 'IT_과학': 895}
단어:  시장, 총 갯수: 48576, 각 범주에서의 갯수: {'문화': 1619, '지역': 25434, '국제': 322, '정치': 3943, '경제': 13272, '사회': 2239, '스포츠': 798, 'IT_과학': 949}
단어:  경기도, 총 갯수: 43081, 각 범주에서의 갯수: {'문화': 1244, '지역': 24275, '국제': 124, '정치': 2237, '경제': 9084, '사회': 3823, '스포츠': 1743, 'IT_과학': 551}
단어:  경기, 총 갯수: 38828, 각 범주에서의 갯수: {'문화': 1052, '지역': 14922, '국제': 225, '정치': 2242, '경제': 9230, '사회': 7059, '스포츠': 3502, 'IT_과학': 596}
단어:  이날, 총 갯수: 17315, 각 범주에서의 갯수: {'문화': 744, '지역': 7708, '국제': 128, '정치': 1890, '경제': 2320, '사회': 3814, '스포츠': 506, 'IT_과학': 205}
단어:  상황, 총 갯수: 12500, 각 범주에서의 갯수: {'문화': 38

In [116]:
find_common_words_and_remove(num_categories=7, com_counts=300)

상위 100개의 단어 확인
단어:   진행, 총 갯수: 31404, 각 범주에서의 갯수: {'문화': 2177, '지역': 15946, '국제': 87, '정치': 950, '경제': 6309, '사회': 4322, '스포츠': 549, 'IT_과학': 1064}
단어:   관계자, 총 갯수: 22403, 각 범주에서의 갯수: {'문화': 703, '지역': 11929, '국제': 98, '정치': 579, '경제': 4981, '사회': 3040, '스포츠': 302, 'IT_과학': 771}
단어:   대표, 총 갯수: 14587, 각 범주에서의 갯수: {'문화': 1018, '지역': 4415, '국제': 43, '정치': 3202, '경제': 3088, '사회': 1376, '스포츠': 902, 'IT_과학': 543}
단어:   시작, 총 갯수: 13410, 각 범주에서의 갯수: {'문화': 1203, '지역': 5375, '국제': 72, '정치': 696, '경제': 3225, '사회': 1899, '스포츠': 372, 'IT_과학': 568}
총 4개의 단어가 겹칩니다.


In [117]:
from collections import Counter

def count_and_remove_low_occurrence_keywords(train_df, test_df, threshold=3):
    # '키워드' 열의 각 값을 쉼표로 분리하여 리스트로 변환
    train_df['키워드_리스트'] = train_df['키워드'].apply(lambda x: x.split(','))
    test_df['키워드_리스트'] = test_df['키워드'].apply(lambda x: x.split(','))

    # '분류_대분류' 별로 단어들을 추출하고 카운팅
    category_keywords = {category: [] for category in train_df["분류_대분류"].unique()}

    for category in category_keywords.keys():
        words = train_df.loc[train_df["분류_대분류"] == category, "키워드_리스트"].tolist()
        category_keywords[category] = [word.strip() for sublist in words for word in sublist if word.strip()]

    # 각 '분류_대분류' 별로 단어들을 카운팅하고 단어가 threshold 이하로 존재하는 경우를 찾기
    low_occurrence_words = set()
    for category, words in category_keywords.items():
        word_counts = Counter(words)
        low_occurrence_words.update({word for word, count in word_counts.items() if count <= threshold})

    # 단어가 threshold 이하로 존재하는 경우를 '키워드' 열에서 제거
    def remove_low_occurrence_words(keywords):
        return ', '.join([word.strip() for word in keywords.split(',') if word.strip() and word.strip() not in low_occurrence_words])

    train_df['키워드'] = train_df['키워드'].apply(remove_low_occurrence_words)

    # test_df에서도 동일한 단어를 제거
    def remove_low_occurrence_words_from_test(keywords):
        return ', '.join([word.strip() for word in keywords.split(',') if word.strip() and word.strip() not in low_occurrence_words])

    test_df['키워드'] = test_df['키워드'].apply(remove_low_occurrence_words_from_test)

    # 제거 후 각 '분류_대분류' 별로 하위 10개의 단어와 그 갯수를 출력
    for category in category_keywords.keys():
        words = train_df.loc[train_df["분류_대분류"] == category, "키워드"].apply(lambda x: x.split(',')).tolist()
        words = [word.strip() for sublist in words for word in sublist if word.strip()]
        word_counts = Counter(words)
        bottom_keywords = word_counts.most_common()[:-11:-1]
        print(f"분류_대분류: {category}")
        for word, count in bottom_keywords:
            print(f"  단어: {word}, 갯수: {count}")
        print()

In [118]:
# 함수 실행
count_and_remove_low_occurrence_keywords(train_df, test_df, threshold=3)

분류_대분류: 문화
  단어: 똘똘, 갯수: 4
  단어: 민관식, 갯수: 4
  단어: 소강실, 갯수: 4
  단어: 영산수련원, 갯수: 4
  단어: 숙영지, 갯수: 4
  단어: 춤판, 갯수: 4
  단어: 본존불, 갯수: 4
  단어: 공렴, 갯수: 4
  단어: 호저집, 갯수: 4
  단어: 박제가, 갯수: 4

분류_대분류: 지역
  단어: 통행거리, 갯수: 4
  단어: 마케팅지수, 갯수: 4
  단어: 온리원, 갯수: 4
  단어: 택시요금체계, 갯수: 4
  단어: 하태권, 갯수: 4
  단어: 북부도시계획도, 갯수: 4
  단어: 춘천해냄대학, 갯수: 4
  단어: 산업혁명센터, 갯수: 4
  단어: 지휘기, 갯수: 4
  단어: 항로변경, 갯수: 4

분류_대분류: 국제
  단어: 위어, 갯수: 4
  단어: 총기사건, 갯수: 4
  단어: 디자인, 갯수: 4
  단어: 진위현, 갯수: 4
  단어: 장소정체성, 갯수: 4
  단어: 역사문화자산, 갯수: 4
  단어: 진다바드, 갯수: 4
  단어: 성접대, 갯수: 4
  단어: 영웅, 갯수: 4
  단어: 연수단, 갯수: 4

분류_대분류: 정치
  단어: 정태인, 갯수: 4
  단어: 달째, 갯수: 4
  단어: 현영석, 갯수: 4
  단어: 은현, 갯수: 4
  단어: 균형발전책, 갯수: 4
  단어: 보행신호등, 갯수: 4
  단어: 경제수, 갯수: 4
  단어: 용전추, 갯수: 4
  단어: 용인전철추진위원회, 갯수: 4
  단어: 재이첩, 갯수: 4

분류_대분류: 경제
  단어: 요즈마코리아, 갯수: 4
  단어: 위메프오, 갯수: 4
  단어: 봉공, 갯수: 4
  단어: 브리타, 갯수: 4
  단어: 평구로지스, 갯수: 4
  단어: 온비드, 갯수: 4
  단어: 트리뷰하우스, 갯수: 4
  단어: 트리뷰, 갯수: 4
  단어: 배액배, 갯수: 4
  단어: 아크로서울포레스트, 갯수: 4

분류_대분류: 사회
  단어: 통합상담소, 갯수: 4
  단어: 일반상담소, 갯수: 

In [119]:
# '키워드' 열의 값이 공백(' ')인 행의 개수 확인 (train_df)
num_blank_keywords_train = train_df[train_df['키워드'].str.strip() == ''].shape[0]

# '키워드' 열의 값이 공백(' ')인 행의 개수 확인 (test_df)
num_blank_keywords_test = test_df[test_df['키워드'].str.strip() == ''].shape[0]

# 공백인 행의 '분류' 값 출력 (train_df)
if '분류' in train_df.columns:
    blank_keywords_train_categories = train_df[train_df['키워드'].str.strip() == '']['분류']
else:
    blank_keywords_train_categories = "train_df에 '분류' 열이 없습니다."

# 공백인 행의 '분류' 값 출력 (test_df)
if '분류' in test_df.columns:
    blank_keywords_test_categories = test_df[test_df['키워드'].str.strip() == '']['분류']
else:
    blank_keywords_test_categories = "test_df에 '분류' 열이 없습니다."

# 결과 출력
print(f"train_df의 전체 행 개수: {len(train_df)}")
print(f"train_df에서 '키워드' 열의 값이 공백인 행의 개수: {num_blank_keywords_train}")
print(f"train_df에서 '키워드' 열의 값이 공백인 행의 '분류' 값:\n{blank_keywords_train_categories}")

print(f"test_df의 전체 행 개수: {len(test_df)}")
print(f"test_df에서 '키워드' 열의 값이 공백인 행의 개수: {num_blank_keywords_test}")
print(f"test_df에서 '키워드' 열의 값이 공백인 행의 '분류' 값:\n{blank_keywords_test_categories}")

# '키워드' 열의 값이 공백(' ')인 행 드랍 (train_df)
train_df = train_df[train_df['키워드'].str.strip() != '']

# '키워드' 열의 값이 공백(' ')인 행 드랍 (test_df)
test_df = test_df[test_df['키워드'].str.strip() != '']

# 드랍 후 데이터프레임 크기 출력
print(f"드랍 후 train_df의 전체 행 개수: {len(train_df)}")
print(f"드랍 후 test_df의 전체 행 개수: {len(test_df)}")

train_df의 전체 행 개수: 54609
train_df에서 '키워드' 열의 값이 공백인 행의 개수: 7
train_df에서 '키워드' 열의 값이 공백인 행의 '분류' 값:
586      스포츠:농구_배구
2640       스포츠:월드컵
10745           지역
14965     문화:요리_여행
18144      경제:경제일반
30955        사회:날씨
39637        사회:여성
Name: 분류, dtype: object
test_df의 전체 행 개수: 23405
test_df에서 '키워드' 열의 값이 공백인 행의 개수: 0
test_df에서 '키워드' 열의 값이 공백인 행의 '분류' 값:
test_df에 '분류' 열이 없습니다.
드랍 후 train_df의 전체 행 개수: 54602
드랍 후 test_df의 전체 행 개수: 23405


In [120]:
# '키워드_리스트' 열 제거 (train_df)
train_df = train_df.drop(columns=['키워드_리스트'])

# '키워드_리스트' 열 제거 (test_df)
test_df = test_df.drop(columns=['키워드_리스트'])

In [121]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54602 entries, 0 to 54608
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      54602 non-null  object
 1   분류      54602 non-null  object
 2   제목      54602 non-null  object
 3   키워드     54602 non-null  object
 4   분류_대분류  54602 non-null  object
dtypes: object(5)
memory usage: 2.5+ MB


In [122]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23405 entries, 0 to 23404
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      23405 non-null  object
 1   제목      23405 non-null  object
 2   키워드     23405 non-null  object
dtypes: object(3)
memory usage: 731.4+ KB


In [123]:
train_df.head()

,ID,분류,제목,키워드,분류_대분류
0,TRAIN_00000,문화:전시_공연,"용인문화재단, 인문학 콘서트 ‘당신이 모르는 뮤지컬 이야기Ⅳ’ 개최","용인문화재단, 뮤지컬, 이야기, 개최, 뮤지컬, 이야기, 용인문화재단, 용인문화재단...",문화
1,TRAIN_00001,지역,"용인 농촌테마파크, 7~8월 단체체험객 체험료 지원","용인, 농촌, 테마파크, 단체, 지원, 체험일, 기준, 용인시통합예약사이트, 접수,...",지역
2,TRAIN_00002,지역,"용인시, 노후주택 에너지 성능 개선 신청 18일까지 연장","용인시, 주택, 에너지, 개선, 신청, 연장, 용인시청, 건축물, 환경친화적, 개선...",지역
3,TRAIN_00003,지역,"수원 용인 고양시,‘특례시’로 지정 도시경쟁력 증가 기대","수원, 고양시, 특례시, 지정, 증가, 도내, 인구, 수원, 고양, 특례시, 지정,...",지역
4,TRAIN_00004,국제,"용인시, 스페인 미국 국제명예자문관 위촉 대외홍보 지원 역할","용인시, 스페인, 미국, 국제, 명예, 자문관, 위촉, 역할, 홍보, 지원, 권태면...",국제


.